In [1]:
import os 
import glob

In [2]:
import pandas as pd
import nibabel as nb 
import numpy as np 

In [3]:
# define paths 
d="output" 
prefix="example__bs"
suffix="_est_parameter.nii.gz"
ext=".nii.gz"

In [4]:
# get files 
files = glob.glob(d+"/"+prefix + "*" + suffix)
assert files 

In [7]:
# get segmentation (we compute CoV over one kidney) 
segf = "seg_generated.nii.gz"
assert os.path.exists(segf)

In [8]:
# get all the labels 
seg = nb.load(segf).get_fdata()
labels = np.unique(seg)
labels = [l for l in labels if l!=0]

In [16]:
# create array to hold pandas dataframes
dfs = []

In [17]:
# load files in numpy 
for label in labels:
    for i, f in enumerate(sorted(files)): 
        # load file 
        im = nb.load(f).get_fdata()
    
        # select masked area
        array = im[seg==label]

    
        # assign to pandas dataframe 
        df = pd.DataFrame({'parameter_value':array})
        df = df.assign(label=label,iteration=i)
        dfs.append(df)
    
    
        

In [18]:
dfs = pd.concat(dfs)

In [19]:
dfs

,parameter_value,label,iteration
0,10.177619,1.0,0
1,-5.566085,1.0,0
2,-4.722335,1.0,0
3,-3.051506,1.0,0
4,-1.356475,1.0,0
...,...,...,...
2236411,-2.269283,1.0,9
2236412,3.366333,1.0,9
2236413,-5.883651,1.0,9
2236414,-7.386290,1.0,9


In [22]:
# let's compute CoV for each label separately 
CoV = df.groupby('label').agg(
    mean=('parameter_value', 'mean'),
    std=('parameter_value', 'std'),
    cv=('parameter_value', lambda x: 100*np.std(x, ddof=0) / np.mean(x))
).reset_index()


In [23]:
CoV 

,label,mean,std,cv
0,1.0,8.054333,17.752283,220.406554
